In [ ]:
# !pip install requests
# !pip install Pillow
# !pip install deepface
# !pip install pandas

In [ ]:
## THIS VERSION USES DATAFRAMES, BUT IT DOESNT WORK RIGHT

# import os
# import numpy as np
# from deepface import DeepFace
# import pandas as pd
# import requests
# from PIL import Image
# from io import BytesIO
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm import tqdm
# import threading

# # Read CSV file
# df = pd.read_csv("input.csv")

# # Separate dataframes for male and female
# df_male = df[df["Gender"] == "Male"]
# df_female = df[df["Gender"] == "Female"]

# dir_male = "images/male/"
# dir_female = "images/female/"

# # Create directories if they don't exist
# os.makedirs(dir_male, exist_ok=True)
# os.makedirs(dir_female, exist_ok=True)

# # Counters for male and female names
# male_counter = 0
# female_counter = 0

# # Locks for thread-safe counter increments
# male_lock = threading.Lock()
# female_lock = threading.Lock()

# # Prompt the user for the acceptable age range
# min_age = int(input("Enter the minimum acceptable age: "))
# max_age = int(input("Enter the maximum acceptable age: "))


# def process_image(gender):
#     global male_counter
#     global female_counter

#     while True:
#         headers = {
#             "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
#         }
#         response = requests.get(
#             "https://www.thispersondoesnotexist.com", headers=headers)

#         if response.status_code == 200 and "image" in response.headers["Content-Type"]:
#             img = Image.open(BytesIO(response.content))

#             # Crop the image
#             width, height = img.size
#             img = img.crop((20, 20, width - 20, height - 20))

#             # Convert the PIL Image to a numpy array
#             img_np = np.array(img)

#             try:
#                 # Predict gender and age using DeepFace
#                 predictions = DeepFace.analyze(img_np, actions=['gender', 'age'])

#                 # Determine the gender based on the higher probability
#                 predicted_gender = "Man" if predictions[0]['gender']['Man'] > predictions[0]['gender']['Woman'] else "Woman"

#                 # Check if the predicted age is within the acceptable range
#                 predicted_age = predictions[0]['age']
#                 if min_age <= predicted_age <= max_age and predicted_gender == gender:
#                     # Save image in the appropriate directory
#                     if predicted_gender == "Man" and male_counter < len(df_male):
#                         with male_lock:
#                             file_name = df_male.iloc[male_counter]["Name"]
#                             full_file_name = dir_male + file_name + ".jpg"
#                             male_counter += 1
#                         img.save(full_file_name)
#                         print(
#                             f"Image {file_name} is predicted to be a {predicted_gender} of age {predicted_age} and saved to {full_file_name}"
#                         )
#                         break
#                     elif predicted_gender == "Woman" and female_counter < len(df_female):
#                         with female_lock:
#                             file_name = df_female.iloc[female_counter]["Name"]
#                             full_file_name = dir_female + file_name + ".jpg"
#                             female_counter += 1
#                         img.save(full_file_name)
#                         print(
#                             f"Image {file_name} is predicted to be a {predicted_gender} of age {predicted_age} and saved to {full_file_name}"
#                         )
#                         break
#             except Exception as e:
#                 print(f"An error occurred while processing the image: {e}")
#         else:
#             print(
#                 f"Failed to download image. Status code: {response.status_code}, Content-Type: {response.headers['Content-Type']}"
#             )



# # Prompt the user for the number of threads
# num_threads = int(input("How many threads do you want to run? "))


# # Use ThreadPoolExecutor to process images in parallel
# with ThreadPoolExecutor(max_workers=num_threads) as executor:
#     tasks = [executor.submit(process_image, gender) for gender in ["Man"]*len(df_male) + ["Woman"]*len(df_female)]

#     # Use tqdm with as_completed to track progress
#     for task in tqdm(as_completed(tasks), total=len(tasks)):
#         task.result()

---

# Get names for personas

In [ ]:
import pandas as pd
import random

# Load the data
data = pd.read_csv('names.csv')

# Get inputs
num_males = int(input("How many MALE names do you want?"))
num_females = int(input("How many FEMALE names do you want?"))

# Generate male names
male_firstnames = data['firstnames_male'].dropna().tolist()
lastnames = data['lastnames'].dropna().tolist()
male_names = [f"{random.choice(male_firstnames)} {random.choice(lastnames)},Male" for _ in range(num_males)]

# Generate female names
female_firstnames = data['firstnames_female'].dropna().tolist()
female_names = [f"{random.choice(female_firstnames)} {random.choice(lastnames)},Female" for _ in range(num_females)]

# Combine and shuffle names
all_names = male_names + female_names
random.shuffle(all_names)

# Write to CSV
with open('input.csv', 'w') as f:
    f.write("Name,Gender\n")
    for name in all_names:
        f.write(f"{name}\n")

# Get images for personas

In [ ]:
import os
import csv
import numpy as np
from deepface import DeepFace
import requests
from PIL import Image
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Read CSV file
with open('input.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header
    data = list(reader)

# Separate data for male and female
male_data = [row for row in data if row[1] == "Male"]
female_data = [row for row in data if row[1] == "Female"]

dir_male = "images/male/"
dir_female = "images/female/"

# Create directories if they don't exist
os.makedirs(dir_male, exist_ok=True)
os.makedirs(dir_female, exist_ok=True)

# Counters for male and female names
male_counter = 0
female_counter = 0

# Locks for thread-safe counter increments
male_lock = threading.Lock()
female_lock = threading.Lock()

# Prompt the user for the acceptable age range
min_age = int(input("Enter the minimum acceptable age: "))
max_age = int(input("Enter the maximum acceptable age: "))

def process_image(gender):
    global male_counter
    global female_counter

    while True:
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        }
        response = requests.get(
            "https://www.thispersondoesnotexist.com", headers=headers)

        if response.status_code == 200 and "image" in response.headers["Content-Type"]:
            img = Image.open(BytesIO(response.content))

            # Crop the image
            width, height = img.size
            img = img.crop((20, 20, width - 20, height - 20))

            # Convert the PIL Image to a numpy array
            img_np = np.array(img)

            try:
                # Predict gender and age using DeepFace
                predictions = DeepFace.analyze(img_np, actions=['gender', 'age'])

                # Determine the gender based on the higher probability
                predicted_gender = "Man" if predictions[0]['gender']['Man'] > predictions[0]['gender']['Woman'] else "Woman"

                # Check if the predicted age is within the acceptable range
                predicted_age = predictions[0]['age']
                if min_age <= predicted_age <= max_age and predicted_gender == gender:
                    # Save image in the appropriate directory
                    if predicted_gender == "Man" and male_counter < len(male_data):
                        with male_lock:
                            file_name = male_data[male_counter][0]
                            full_file_name = dir_male + file_name + ".jpg"
                            male_counter += 1
                        img.save(full_file_name)
                        print(
                            f"Image {file_name} is predicted to be a {predicted_gender} of age {predicted_age} and saved to {full_file_name}"
                        )
                        break
                    elif predicted_gender == "Woman" and female_counter < len(female_data):
                        with female_lock:
                            file_name = female_data[female_counter][0]
                            full_file_name = dir_female + file_name + ".jpg"
                            female_counter += 1
                        img.save(full_file_name)
                        print(
                            f"Image {file_name} is predicted to be a {predicted_gender} of age {predicted_age} and saved to {full_file_name}"
                        )
                        break
            except Exception as e:
                print(f"An error occurred while processing the image: {e}")
        else:
            print(
                f"Failed to download image. Status code: {response.status_code}, Content-Type: {response.headers['Content-Type']}"
            )

# Prompt the user for the number of threads
num_threads = int(input("How many threads do you want to run? "))

# Use ThreadPoolExecutor to process images in parallel
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    tasks = [executor.submit(process_image, gender) for gender in ["Man"]*len(male_data) + ["Woman"]*len(female_data)]

    # Use tqdm with as_completed to track progress
    for task in tqdm(as_completed(tasks), total=len(tasks)):
        task.result()